## Ejecutamos el script para obtener las transcripciones

#### Importar transcripcines

In [63]:
import os

# Ejecutamos el script crear_transcripciones.py
%run /home/jovyan/work/scripts/crear_transcripciones.py

# Para listar archivos en el directorio actual
# print("Archivos en el directorio actual:")
# print(os.listdir('/home/jovyan/work'))


Transcripciones extraidas correctamente y guardadas en la carpeta 'transcripciones' (en la carpeta de notebooks).


In [64]:
import os
import re
from bs4 import BeautifulSoup
from num2words import num2words

def clean_text(text):
    # Eliminar etiquetas HTML
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    # Eliminar los sonidos externos entre corchetes
    text = re.sub(r'\[.*?\]', '', text)
    # Eliminar los sonidos externos marcados con *
    text = re.sub(r'\*.*?\*', '', text)
    # Eliminar los sonidos externos entre paréntesis
    text = re.sub(r'\(.*?\)', '', text)
    # Convertir números a texto en español
    text = re.sub(r'\d+', lambda x: num2words(int(x.group()), lang='es'), text)
    # Eliminar caracteres especiales
    text = re.sub(r'[^A-Za-záéíóúüñÁÉÍÓÚÜÑ\s]', '', text)
    # Eliminar múltiples espacios en blanco
    text = re.sub(r'\s+', ' ', text).strip()
    return text

input_directory = '/home/jovyan/work/data/transcripciones'
output_directory = '/home/jovyan/work/data/cleaned'

# Guardamos los archivos
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    
for filename in os.listdir(input_directory):
    if filename.endswith('.txt'):
        with open(os.path.join(input_directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            cleaned_text = clean_text(text)
            with open(os.path.join(output_directory, filename), 'w', encoding='utf-8') as out_file:
                out_file.write(cleaned_text)


## Normalizacion

In [65]:
import os
import re

def normalize(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales pero conservar letras acentuadas, espacios y signos de puntuación básicos
    text = re.sub(r'[^a-záéíóúüñ\s.,]', '', text)
    return text

input_directory = '/home/jovyan/work/data/cleaned'
output_directory = '/home/jovyan/work/data/normalized'

# Guardamos los archivos
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for filename in os.listdir(input_directory):
    if filename.endswith('.txt'):
        with open(os.path.join(input_directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            normalized_text = normalize(text)
            with open(os.path.join(output_directory, filename), 'w', encoding='utf-8') as out_file:
                out_file.write(normalized_text)


## Corrección ortográfica

In [66]:
# from spellchecker import SpellChecker

# spell = SpellChecker(language='es')

# def correct_spelling(text):
#     corrected_text = []
#     for word in text.split():
#         corrected_word = spell.correction(word)
#         if corrected_word is not None:
#             corrected_text.append(corrected_word)
#     return ' '.join(corrected_text)

# input_directory = '/home/jovyan/work/data/normalized'
# output_directory = '/home/jovyan/work/data/corrected'

# if not os.path.exists(output_directory):
#     os.makedirs(output_directory)

# for filename in os.listdir(input_directory):
#     if filename.endswith('.txt'):
#         with open(os.path.join(input_directory, filename), 'r', encoding='utf-8') as file:
#             text = file.read()
#             spelled_text = correct_spelling(text)
#             with open(os.path.join(output_directory, filename), 'w', encoding='utf-8') as out_file:
#                 out_file.write(spelled_text)


## Tokenizacion

In [67]:
import os
import spacy

# spaCy en español
nlp = spacy.load("es_core_news_sm")

def tokenize(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    return tokens

# input_directory = '/home/jovyan/work/data/corrected'
input_directory = '/home/jovyan/work/data/normalized'
output_directory = '/home/jovyan/work/data/tokenized'

# Guardamos los archivos tokenizados
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for filename in os.listdir(input_directory):
    if filename.endswith('.txt'):
        with open(os.path.join(input_directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            tokens = tokenize(text)
            with open(os.path.join(output_directory, filename), 'w', encoding='utf-8') as out_file:
                out_file.write(' '.join(tokens))


## Stopwords

In [68]:
import os
import spacy
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

# spaCy en español
nlp = spacy.load("es_core_news_sm")

def remove_stopwords(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if token.text.lower() not in stop_words]
    return tokens

input_directory = '/home/jovyan/work/data/tokenized'
output_directory = '/home/jovyan/work/data/no_stopwords'

# Guardamos los archivos sin las stopwords
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for filename in os.listdir(input_directory):
    if filename.endswith('.txt'):
        with open(os.path.join(input_directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            tokens = remove_stopwords(text)
            with open(os.path.join(output_directory, filename), 'w', encoding='utf-8') as out_file:
                out_file.write(' '.join(tokens))


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Lemmatización

In [69]:
import os
import spacy

# spaCy en español
nlp = spacy.load("es_core_news_sm")

def lemmatize_with_context(text):
    doc = nlp(text)
    lemmatized_tokens = []

    for token in doc:
        if token.pos_ in ["NOUN", "PRON", "ADJ", "DET"]:
            lemmatized_tokens.append(token.text)
        else:
            lemmatized_tokens.append(token.lemma_)
    
    lemmatized_text = " ".join(lemmatized_tokens)
    return lemmatized_text

# input_directory = '/home/jovyan/work/data/no_stopwords'
input_directory = '/home/jovyan/work/data/tokenized'
output_directory = '/home/jovyan/work/data/lemmatized'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for filename in os.listdir(input_directory):
    if filename.endswith('.txt'):
        with open(os.path.join(input_directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            lemmatized_text = lemmatize_with_context(text)
            with open(os.path.join(output_directory, filename), 'w', encoding='utf-8') as out_file:
                out_file.write(lemmatized_text)


## Reconstrucción Gramatical

In [70]:
import os
import spacy

# spaCy en español
nlp = spacy.load("es_core_news_sm")

def reconstruct_grammar(text):
    doc = nlp(text)
    sentences = []
    
    for sent in doc.sents:
        words = []
        for token in sent:
            words.append((token.i, token.text, token.dep_, token.head.i, token.pos_))
        
        # Mantener el orden de las palabras según las dependencias gramaticales
        words_sorted = sorted(words, key=lambda x: x[0])
        
        # Reconstruir la oración manteniendo la coherencia gramatical
        reordered_sentence = " ".join([word[1] for word in words_sorted])
        sentences.append(reordered_sentence)

    return " ".join(sentences)

input_directory = '/home/jovyan/work/data/tokenized'
output_directory = '/home/jovyan/work/data/reconstructed'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for filename in os.listdir(input_directory):
    if filename.endswith('.txt'):
        with open(os.path.join(input_directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            reconstructed_text = reconstruct_grammar(text)
            with open(os.path.join(output_directory, filename), 'w', encoding='utf-8') as out_file:
                out_file.write(reconstructed_text)


## Reorganizamos las frases

In [71]:
import os
import spacy

# Cargar el modelo de lenguaje español de spaCy
nlp = spacy.load("es_core_news_sm")

# Función para reorganizar las palabras en frases coherentes
def reorganize_sentences(text):
    doc = nlp(text)
    sentences = []

    for sent in doc.sents:
        words = []
        for token in sent:
            if token.pos_ in ["NOUN", "VERB", "ADJ", "ADV", "PRON", "DET", "ADP", "CONJ", "CCONJ", "SCONJ"]:
                words.append((token.i, token.text, token.dep_))
        
        # Mantener el orden de las palabras según las dependencias gramaticales
        words_sorted = sorted(words, key=lambda x: x[0])
        reordered_sentence = " ".join([word[1] for word in words_sorted])
        sentences.append(reordered_sentence)

    return " ".join(sentences)

input_directory = '/home/jovyan/work/data/reconstructed'
output_directory = '/home/jovyan/work/data/reestructured'

# Crear el directorio de salida si no existe
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Procesar los archivos en el directorio de entrada
for filename in os.listdir(input_directory):
    if filename.endswith('.txt'):
        with open(os.path.join(input_directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            reorganized_text = reorganize_sentences(text)
            
            # Guardar el texto reorganizado en el directorio de salida
            with open(os.path.join(output_directory, filename), 'w', encoding='utf-8') as out_file:
                out_file.write(reorganized_text)

print("Textos reorganizados y guardados en el directorio 'reestructured'.")


Textos reorganizados y guardados en el directorio 'reestructured'.


## Análisis de Resultados
### Comparación de Textos Originales y Corregidos

In [72]:
import os

# Función para leer los primeros n caracteres de un archivo
def read_first_n_chars(file_path, n):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        return text[:n]

# Función para mostrar la comparación entre el texto original y el corregido
def compare_texts(original, corrected):
    print(f"Original: {original}")
    print(f"Corrected: {corrected}")
    print("="*50)

# Función principal para comparar textos en archivos
def compare_files(original_directory, corrected_directory, num_chars):
    # Obtener los nombres de los archivos en el directorio original
    file_names = [f for f in os.listdir(original_directory) if f.endswith('.txt')]

    # Aplicar la comparación a los archivos de texto
    for file_name in file_names:
        original_path = os.path.join(original_directory, file_name)
        corrected_path = os.path.join(corrected_directory, file_name)
        
        # Verificar que el archivo corregido exista
        if os.path.exists(corrected_path):
            original_text = read_first_n_chars(original_path, num_chars)
            corrected_text = read_first_n_chars(corrected_path, num_chars)
            
            # Comparar los textos
            compare_texts(original_text, corrected_text)
        else:
            print(f"Archivo corregido no encontrado para: {file_name}")

# Directorios de transcripciones originales y corregidas
original_directory = '/home/jovyan/work/data/transcripciones'
corrected_directory = '/home/jovyan/work/data/reestructured'

# Número de caracteres a comparar (puedes cambiar este valor según necesites)
num_chars_to_compare = 100

# Llamada a la función principal
compare_files(original_directory, corrected_directory, num_chars_to_compare)


Original: mi gente de Perú qué tal estáis ayer nos quedamos bueno el otro día nos quedamos eh hostias es el ca
Corrected: mi gente de qué tal ayer nos quedamos bueno el otro día nos quedamos hostias el caballo vale no tiem
Original: [Música] [Música] [Música] más [Música] voy a explicarte todo lo que te extraño te extraño demasiado
Corrected: más a explicarte todo lo que te extraño te extraño demasiado ahora como tan diario yo yo ya juegan a
Original: Estamos en uno de los trenes
más costosos de todo el mundo. Tiene spa, restaurantes, bar, terraza, u
Corrected: en uno de los trenes más costosos de todo el mundo tiene spa bar terraza una suite digna de un hotel
Original: Qué tal eh sazonado con tinta con el sabor de las noticias del día de hoy a la madre güey hay carril
Corrected: qué tal sazonado con tinta con el sabor de las noticias del día de hoy a la madre güey carriles tú a
Original: Qué pasa chicos eh estoy aquí un poco a oscuras y escondido porque bueno para poneros en contexto 